* 머신러닝 알고리즘을 적용하기 전에 거치는 데이터 전처리 과정
* 머신러닝 알고리즘이 데이터에 기반하기 때문에 결과를 좌지우지하는 중요한 단계
* 결손값 대체/문자열 인코딩

## 데이터 인코딩

**레이블 인코딩** - 카테고리 피처를 코드형 숫자 값으로 변환

In [2]:
from sklearn.preprocessing import LabelEncoder

items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','믹서','믹서']

# LabelEncoder를 객체로 생성한 후, fit()과 transform()으로 레이블 인코딩 수행
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값:', labels)

인코딩 변환값: [0 1 4 5 3 3 2 2]


In [3]:
print('인코딩 클래스:', encoder.classes_)

인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


In [4]:
print('디코딩 원본값:', encoder.inverse_transform([4,5,1,0,2,2,3]))

디코딩 원본값: ['전자레인지' '컴퓨터' '냉장고' 'TV' '믹서' '믹서' '선풍기']


* 몇몇 ML 알고리즘은 숫자 값의 경우 크고 작음에 대한 특성이 작용하기 때문에 예측 성능이 떨어지는 경우가 발생
* 그래서 선형 회귀와 같은 ML 알고리즘에 적용 불가
* 트리 계열의 ML 알고리즘은 가능
* 이러한 문제점을 해결해주는 것은 원-핫 인코딩(One-Hot Encoding)

**원-핫 인코딩(One-Hot Encoding)** - 피처 값의 유형에 따라 새로운 피처를 추가해 고유 값에 해당하는 칼럼에만 1을 표시, 나머지 칼럼에는 0을 표시

In [7]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','믹서','믹서']

# 먼저 숫자 값으로 변환을 위해 LabelEncoder로 변환
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

# 2차원 데이터로 변환
labels = labels.reshape(-1,1)

# 원-핫 인코딩을 적용
oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원
(8, 6)


**Pandas의 get_dummies()** - 숫자형 변환 없이 바로 원-핫 인코딩으로 변환

In [8]:
import pandas as pd

df = pd.DataFrame({'items' : ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','믹서','믹서']})
pd.get_dummies(df)

,items_TV,items_냉장고,items_믹서,items_선풍기,items_전자레인지,items_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## 피처 스케일링과 정규화

피처 스케일링 : 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업
* 표준화(Standardization): 평균이 0이고 분산이 1인 가우시안 정규 분포를 가진 값으로 변환
* 정규화(Normalization): 서로 다른 피처의 크기를 통일하기 위해 크기를 변환. 즉, 개별 데이터의 크기를 모두 똑같은 단위로 변경

**StandardScaler** - 표준화

*서포트 벡터 머신(Support Vector Machine), 선형회귀(Linear Regression), 로지스틱 회귀(Logistic Regression)는 데이터가 가우시안 분포를 가지고 있다고 가정하고 구현됐기 때문에 사전에 표준화를 적용하는 것이 예측 성능 향상에 중요한 요소이다.*

In [11]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

print('feature의 평균 값:\n',iris_df.mean())
print('\n feature의 분산 값:\n',iris_df.var())

feature의 평균 값:
 sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

 feature의 분산 값:
 sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [14]:
# StandardScaler

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform() 시 Numpy ndarray로 반환됨
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print('feature의 평균 값:\n',iris_df_scaled.mean())
print('\n feature의 분산 값:\n',iris_df_scaled.var())

feature의 평균 값:
 sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

 feature의 분산 값:
 sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


**MinMaxScaler** - 정규화

데이터값을 0과 1사이의 범위 값으로 변환(음수 값이 있으면 -1에서 1값으로 변환)

In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform() 시 Numpy ndarray로 반환됨
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print('feature의 평균 값:\n',iris_df_scaled.mean())
print('\n feature의 분산 값:\n',iris_df_scaled.var())

print('\n feature의 최솟값:\n',iris_df_scaled.min())
print('\n feature의 최댓값:\n',iris_df_scaled.max())

feature의 평균 값:
 sepal length (cm)    0.428704
sepal width (cm)     0.440556
petal length (cm)    0.467458
petal width (cm)     0.458056
dtype: float64

 feature의 분산 값:
 sepal length (cm)    0.052908
sepal width (cm)     0.032983
petal length (cm)    0.089522
petal width (cm)     0.100869
dtype: float64

 feature의 최솟값:
 sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

 feature의 최댓값:
 sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


## 학습 데이터와 테스트 데이터의 스케일링 변환 시 유의점

In [18]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터는 0부터 10, 테스트 데이터는 0부터 5
# Scaler 클래스의 fit(), transform()은 2차원 이상 데이터만 가능하므로 reshape으로 차원 변경
train_array = np.arange(0,11).reshape(-1,1)
test_array = np.arange(0,6).reshape(-1,1)

In [19]:
## 학습 데이터인 train_array부터 변환

# MinMaxScaler에 별도의 feature_range 파라미터 값을 지정하지 않으면 0~1 값으로 변환
scaler = MinMaxScaler()

# fit()하게 되면 train_array 데이터의 최솟값이 0, 최댓값이 10으로 설정
scaler.fit(train_array)

# 1/10 scale로 train_array 데이터 변환. 원본이 10 -> 1 로 변환
train_scaled = scaler.transform(train_array)

print('원본 train_array 데이터:', np.round(train_array.reshape(-1),2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1),2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [20]:
## 테스트 데이터인 test_array 변환

# fit()하게 되면 train_array 데이터의 최솟값이 0, 최댓값이 5으로 설정
scaler.fit(test_array)

# 1/5 scale로 test_array 데이터 변환. 원본이 5 -> 1 로 변환
test_scaled = scaler.transform(test_array)

print('원본 test_array 데이터:', np.round(test_array.reshape(-1),2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1),2))

원본 test_array 데이터: [0 1 2 3 4 5]
Scale된 test_array 데이터: [0.  0.2 0.4 0.6 0.8 1. ]


=> 학습 데이터와 테스트 데이터의 스케일링이 맞지 않음

테스트 데이터에 다시 fit()을 적용해선 안되며 학습 데이터로 이미 fit()이 적용된 Scaler 객체를 이용해 transform()으로 변환해야 한다.

In [21]:
scaler = MinMaxScaler()

scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print('원본 train_array 데이터:', np.round(train_array.reshape(-1),2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1),2))

# test 데이터는 반드시 fit()을 호출하지 않고 transform()만으로 변환
test_scaled = scaler.transform(test_array)
print('원본 test_array 데이터:', np.round(test_array.reshape(-1),2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1),2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
원본 test_array 데이터: [0 1 2 3 4 5]
Scale된 test_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5]


* 가능하다면 전체 데이터의 스케일링 변환을 적용한 뒤 학습과 테스트 데이터 분리
* 여의치 않다면 테스트 데이터 변환 시에는 학습 데이터로 이미 fit()된 Scaler 객체를 이용해 transform() 변환

*fit_transform() 사용 시 주의*